In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az

In [2]:
data = []
with open('data.txt' , 'r') as f:
    for line in f.readlines():
        data.append(line.strip('\n'))

In [3]:
print(data)

['## Data', 'lsat = Dict{Symbol, Any}(', '  :cumulative Frequency =>', '    [3, 9, 11, 22, 23, 24, 27, 31, 32, 40, 40, 56, 56, 59, 61, 76, 86, 115, 129,', '     210, 213, 241, 256, 336, 352, 408, 429, 602, 613, 674, 702, 1000],', '  :response =>', '    [0 0 0 0 0', '     0 0 0 0 1', '     0 0 0 1 0', '     0 0 0 1 1', '     0 0 1 0 0', '     0 0 1 0 1', '     0 0 1 1 0', '     0 0 1 1 1', '     0 1 0 0 0', '     0 1 0 0 1', '     0 1 0 1 0', '     0 1 0 1 1', '     0 1 1 0 0', '     0 1 1 0 1', '     0 1 1 1 0', '     0 1 1 1 1', '     1 0 0 0 0', '     1 0 0 0 1', '     1 0 0 1 0', '     1 0 0 1 1', '     1 0 1 0 0', '     1 0 1 0 1', '     1 0 1 1 0', '     1 0 1 1 1', '     1 1 0 0 0', '     1 1 0 0 1', '     1 1 0 1 0', '     1 1 0 1 1', '     1 1 1 0 0', '     1 1 1 0 1', '     1 1 1 1 0', '     1 1 1 1 1],', '  :N => 1000', ')', '']


In [4]:
choices = []
print()
for student in data[6 : len(data) - 3]:
    choice = []
    student_rev = student[::-1]
    student_split = student.split(' ')
    for c in student_split[-5 : ]:
        if '[' in c:
            choice.append(int(c[1]))
        elif ']' in c:
            choice.append(int(c[0]))
        else:
            choice.append(int(c))
    choices.append(choice)
        

In [5]:
cumulative = []

cum1 = data[3].strip(' ')
cum2 = data[4].strip(' ')[:-1]
cum1 = cum1.strip('[')
cum2 = cum2.strip(']')
cum = cum1 + cum2
cum_split = cum.split(',')

for c in cum_split:
    cumulative.append(int(c))

Rasch Model

$\Pr(X_{ni}=1) = \frac{e^{\beta_n - \delta_i}}{1 + e^{\beta_n - \delta_i}}$

$\beta_n$ = ability of student n

$\delta_i$ = difficulty of item i

https://towardsdatascience.com/a-bayesian-approach-to-rasch-models-item-response-theory-cc08805cbb37

In [6]:
with pm.Model() as model:
    
    # prior on alpha
    alpha = pm.Normal('alpha', mu=0, sigma=100 , shape = 5)

    # prior on beta
    beta = pm.Flat('beta')

    # prior on theta
    theta = pm.Normal('theta' , mu = 0 , sigma = 1 , shape = 1000)

    p = np.zeros((1000 , 5))
    for i in range(1000):
        for j in range(5):
            p[i][j] = pm.math.invlogit(alpha[j] + beta * theta[i])

    r = pm.Bernoulli('r' , p = p , observed = choices)

    trace_100 = pm.sample(100,return_inferencedata=True)

ValueError: setting an array element with a sequence.